In [20]:
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import pickle
import json
from database.comet import Comet
from processor.processor import Processor as p
from time import sleep
import matplotlib.pyplot as plt
import math
from backtester.entry import Entry
from backtester.exit_strats import ExitStrats

In [21]:
comet = Comet()

In [22]:
comet.connect()
prices = comet.retrieve(f"alpha_prices")
comet.disconnect()

In [23]:
prices["crypto"].unique()

array(['ADA', 'BTC', 'CAKE', 'DOGE', 'ETH', 'SHIB', 'XTZ', 'XRP', 'LUNA',
       'AVAX', 'LTC', 'DOT', 'MATIC'], dtype=object)

In [24]:
symbols = [
            'ADA', 
            'BTC',
           'DOGE', 
           'ETH', 
           'SHIB', 
           'LUNA',
           'MATIC'
           'AVAX', 
            'LTC', 
            'DOT'
        ]

In [ ]:
api = "coinbase"
retrack_days = [1,3,7,14]
signals = range(0,60,10)
reqs = range(10,60,10)
entry_strats = ["standard","signal_based","parameter_defined","all"]
exit_strats = [
                "due_date","hold","adaptive_due_date",
    "adaptive_hold"]
trades = []
market = prices.pivot_table(index="date",columns="crypto",values="close").reset_index()
market = p.column_date_processing(market)
market = market.fillna(method="ffill")
start = datetime(2021,1,1)
end = datetime(2021,12,31)
for conservative in tqdm([True,False],desc="conservative"):
    for value in tqdm([True,False],desc="value"):
        for rt in tqdm(retrack_days,desc="retrack_days"):
            sim = market.melt(id_vars="date").copy()
            ns = []
            for crypto in sim["crypto"].unique():
                crypto_sim = sim[sim["crypto"]==crypto].copy()
                crypto_sim.sort_values("date",inplace=True)
                crypto_sim["signal"] = crypto_sim["value"].pct_change(rt)
                crypto_sim["velocity"] = crypto_sim["signal"].pct_change(rt)
                crypto_sim["inflection"] = crypto_sim["velocity"].pct_change(rt)
                crypto_sim["p_sign_change"] = [row[1]["velocity"] * row[1]["inflection"] < 0 for row in crypto_sim.iterrows()]
                ns.append(crypto_sim)
            final = pd.concat(ns)
            final = final[(final["date"] < end) & (final["crypto"].isin([x.lower() for x in symbols]))]
            for entry_strat in tqdm(entry_strats,desc="entry"):
                for exit_strat in exit_strats:
                    for s in signals:
                        for r in reqs:
                            signal = float(s/100)
                            req = float(r/100)
                            date = start
                            while date < end:
                                try:
                                    match entry_strat:
                                        case "standard":
                                            offerings = Entry.standard(final,date,signal,value,conservative)
                                        case "signal_based":
                                            offerings = Entry.signal_based(final,date,signal,value,conservative)
                                        case "parameter_defined":
                                            offerings = Entry.parameter_defined(final,date,signal,value,conservative)
                                        case "all":
                                            offerings = Entry.all(final,date,signal,value,conservative)
                                        case _:
                                            offerings = pd.DataFrame([{}])
                                    if offerings.index.size < 1:
                                        date = date + timedelta(days=1)
                                    else:
                                        trade = offerings.iloc[0]
                                        match exit_strat:
                                            case "due_date":
                                                trade = ExitStrats.due_date(final,trade,rt,req)
                                            case "hold":
                                                trade = ExitStrats.hold(final,trade,rt,req)
                                            case "adaptive_due_date":
                                                trade = ExitStrats.adaptive_due_date(final,trade,rt,req)
                                            case "adaptive_hold":
                                                trade = ExitStrats.adaptive_hold(final,trade,rt,req)
                                            case _:
                                                trade = {}
                                        trade["signal"] = signal
                                        trade["req"] = req
                                        trade["retrack_days"] = rt
                                        trade["value"] = value
                                        trade["conservative"] = conservative
                                        trade["entry_strategy"] = entry_strat
                                        trade["exit_strategy"] = exit_strat
                                        trades.append(trade)
                                        date = trade["sell_date"] + timedelta(days=1)
                                except Exception as e:
                                    print(date,str(e))
                                    date = date + timedelta(days=1)

value:   0%|                                                                                                                                           | 0/2 [00:00<?, ?it/s]

retrack_days:   0%|                                                                                                                                    | 0/4 [00:00<?, ?it/s]


entry:   0%|                                                                                                                                           | 0/4 [00:00<?, ?it/s]E:\projects\comet_venv\comet\backtester\exit_strats.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exits["delta"] = (exits["value"] - bp) / bp
C:\Users\eric\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core

entry:  50%|█████████████████████████████████████████████████████████████████▌                                                                 | 2/4 [00:26<00:26, 13.36s/it]


entry:  75%|██████████████████████████████████████████████████████████████████████████████████████████████████▎                                | 3/4 [01:02<00:23, 23.53s/it]


entry: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:21<00:00, 20.49s/it]


retrack_days:  25%|███████████████████████████████                                                                                             | 1/4 [01:22<04:07, 82.55s/it]


entry:   0%|                                                                                                                                           | 0/4 [00:00<?, ?it/s]

In [ ]:
t = pd.DataFrame(trades)

In [ ]:
analysis = []
for conservative in [True,False]:
    for value in [True,False]:
        for entry_strat in entry_strats:
            for exit_strat in tqdm(exit_strats,desc="exit"):
                for rt in retrack_days:
                    for s in tqdm(signals):
                        for r in reqs:
                            try:
                                signal = float(s/100)
                                req = float(r/100)
                                iteration_trades = t[(t["signal"]==signal) 
                                                     & (t["req"]==req) 
                                                     & (t["retrack_days"]==rt) 
                                                     & (t["value"]==value) 
                                                     & (t["conservative"]==conservative)
                                                    & (t["entry_strategy"]==entry_strat)
                                                     & (t["exit_strategy"]==exit_strat)
                                                    ].sort_values("sell_date",ascending=True)
                                initial = 100
                                for delta in iteration_trades["delta"]:
                                     initial = initial * (1+delta)
                                iteration_trades["hpr"] = iteration_trades["sell_date"] - iteration_trades["date"]
                                iteration_trades["days"] = [x.days for x in iteration_trades["hpr"]]
                                days = iteration_trades["days"].mean()
                                analysis.append({"signal":signal,"req":req,"trades":iteration_trades.index.size,"pv":initial,"days":days,"retrack_days":rt,"value":value,"conservative":conservative,"entry_strategy":entry_strat,"exit_strategy":exit_strat})
                            except Exception as e:
                                print(str(e))

In [ ]:
a = pd.DataFrame(analysis)
a.sort_values("pv",ascending=False).head(30)

In [ ]:
a[a["exit_strategy"]=="adaptive_hold"].sort_values("pv",ascending=False).head(10)

In [ ]:
api = "alpha"
goods = a.sort_values("pv",ascending=False).head(10).iloc[0]
current_trades = t[(t["signal"]==goods["signal"].item())
                       & (t["req"]==goods["req"].item())
                        & (t["retrack_days"]==goods["retrack_days"].item())
                          & (t["value"]==goods["value"].item())
                            & (t["conservative"]==goods["conservative"].item())
                   & (t["entry_strategy"]==goods["entry_strategy"])
                   & (t["exit_strategy"]==goods["exit_strategy"])
                  ]
current_trades.sort_values("sell_date",inplace=True)
if current_trades.index.size > 1:
    viz = []
    row = current_trades.iloc[0]
    pv = 100
    start_date = row["date"]
    symbol = row["crypto"]
    amount = float(pv/row["buy_price"])
    end_date = row["sell_date"]
    pv2 = amount * row["sell_price"]
    viz.append({"date":start_date,"crypto":symbol,"amount":amount})
    viz.append({"date":end_date,"crypto":symbol,"amount":amount})
    track_date = start_date
    while track_date < end_date - timedelta(days=1):
        track_date = track_date + timedelta(days=1)
        viz.append({"date":track_date,"crypto":symbol,"amount":amount})
    for i in range(1,current_trades.index.size-1):
        row = current_trades.iloc[i]
        symbol = current_trades.iloc[i]["crypto"]
        start_date = row["date"]
        pv = pv2
        amount =  pv /row["buy_price"]
        viz.append({"date":start_date,"crypto":symbol,"amount":amount})
        track_date = start_date
        end_date = row["sell_date"]
        while track_date < end_date:
            track_date = track_date + timedelta(days=1)
            viz.append({"date":track_date,"crypto":symbol,"amount":amount})
        pv2 = amount * row["sell_price"]
        viz.append({"date":end_date,"crypto":symbol,"amount":amount})
    window = pd.DataFrame(viz)
    example = final.merge(window,how="left",on=["date","crypto"])
    example = example.dropna().sort_values("date")
    example["actual"] = example["amount"] * example["value"]
    example = example.merge(market[["date","btc"]],on="date",how="left")
    example["bench_delta"] = (example["btc"] - example["btc"].iloc[0]) / example["btc"].iloc[0]
    example["actual_delta"] = (example["actual"] - example["actual"].iloc[0]) / example["actual"].iloc[0]
#     if example["actual"].max() < 10000:
    plt.plot(example["date"],example["actual_delta"])
    plt.plot(example["date"],example["bench_delta"])
plt.show()

In [ ]:
symbol_analysis = []
for symbol in current_trades["crypto"].unique():
    symbol_trades = current_trades[current_trades["crypto"]==symbol]
    trades = symbol_trades.index.size
    initial = 1
    for delta in symbol_trades["delta"]:
        initial = initial * 1+delta
    symbol_return = initial
    symbol_trades["hpr"] = symbol_trades["sell_date"] - symbol_trades["date"]
    symbol_trades["days"] = [x.days for x in symbol_trades["hpr"]]
    symbol_analysis.append({"symbol":symbol,"return":initial,"trades":trades,"hpr":symbol_trades["days"].mean()})

In [ ]:
initial

In [ ]:
pd.DataFrame(symbol_analysis).sort_values("return",ascending=False)

In [ ]:
initial = 100
for delta in current_trades["delta"]:
    initial = initial * (1+delta)
    print(initial,1+delta)
initial

In [ ]:
current_trades

In [ ]:
nice= pd.DataFrame(analysis)
nice.sort_values("pv",ascending=False).head(30)